In [1]:
from datasets.stored_dataframe import CapgMyoDataModule, MyoArmbandDataModule, NinaProDataModule
from models import Classifier, OriginalModel
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torch
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchvision.transforms import Compose, ToTensor, Normalize
from models import LightningXGBClassifier
import os
from definitions import MODELS_FOLDER, ROOT_DIR
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## Chinese Experiment

In [2]:
pl.seed_everything(42, workers=True)
lr_lambda=lambda epoch: 1 if epoch<16 else 0.1 if epoch<24 else 0.01
data_module_capgmyo = CapgMyoDataModule(
    batch_size=1000
)
parameters = data_module_capgmyo.get_data_parameters()
model_capgmyo_chinese = OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height'])
lr_scheduler = torch.optim.lr_scheduler.LambdaLR
optimizer = torch.optim.RAdam
classifier = Classifier(model_capgmyo_chinese, optimizer=optimizer, lr_scheduler=lr_scheduler, optim_kwargs={'lr': 0.001, 'weight_decay': 0.0001}, monitor='val_accuracy', lr_lambda=lr_lambda)
logger_capgmyo_chinese = WandbLogger(project="EMG Armband", name="Chinese CapgMyo")
trainer_capgmyo_chinese = pl.Trainer(gpus=-1, max_epochs=28, logger=logger_capgmyo_chinese, accelerator="gpu")

Global seed set to 42


KeyError: 'width'

In [ ]:
trainer_capgmyo_chinese.fit(model=classifier, datamodule=data_module_capgmyo)

In [ ]:
trainer_capgmyo_chinese.test(model=classifier, datamodule=data_module_capgmyo)

In [ ]:
torch.save(model_capgmyo_chinese.state_dict(), os.path.join(MODELS_FOLDER, 'chinese_capgmyo.pt'))

## CapgMyo Experiment

In [ ]:
pl.seed_everything(42, workers=True)
data_module_capgmyo = CapgMyoDataModule(
    batch_size=1000
)
parameters = data_module_capgmyo.get_data_parameters()
model_capgmyo = OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height'])
classifier = Classifier(model_capgmyo, optim_kwargs={'lr': 0.001, 'weight_decay': 0.0001},
                        monitor='val_accuracy', sched_kwargs={'patience': 4, 'mode': 'max'})
logger_capgmyo = WandbLogger(project="EMG Armband", name="CapgMyo")
early_stop_callback = EarlyStopping(monitor='val_accuracy', patience=7, mode='max')
trainer_capgmyo = pl.Trainer(gpus=-1, max_epochs=100, logger=logger_capgmyo, accelerator="gpu", callbacks=[early_stop_callback])

In [ ]:
trainer_capgmyo.fit(model=classifier, datamodule=data_module_capgmyo)

In [ ]:
trainer_capgmyo.test(model=classifier, datamodule=data_module_capgmyo)

In [ ]:
torch.save(model_capgmyo_chinese.state_dict(), os.path.join(MODELS_FOLDER, 'capgmyo.pt'))

## Ninapro Experiment

In [ ]:
pl.seed_everything(42, workers=True)
data_module_ninapro = NinaProDataModule(
    batch_size=1000
)
parameters = data_module_ninapro.get_data_parameters()
model_ninapro = OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height'])
classifier = Classifier(model_ninapro, optim_kwargs={'lr': 0.001, 'weight_decay': 0.0001},
                        monitor='val_accuracy', sched_kwargs={'patience': 4, 'mode': 'max'})
logger_ninapro = WandbLogger(project="EMG Armband", name="NinaPro")
early_stop_callback = EarlyStopping(monitor='val_accuracy', patience=7, mode='max')
trainer_ninapro = pl.Trainer(gpus=-1, max_epochs=100, logger=logger_ninapro, accelerator="gpu", callbacks=[early_stop_callback])

In [ ]:
trainer_ninapro.fit(model=classifier, datamodule=data_module_ninapro)

In [ ]:
trainer_ninapro.test(model=classifier, datamodule=data_module_ninapro)

In [ ]:
torch.save(model_capgmyo_chinese.state_dict(), os.path.join(MODELS_FOLDER, 'ninapro.pt'))

## MyoArmband Experiment

In [ ]:
pl.seed_everything(42, workers=True)
transform = Compose([
    ToTensor(),
    Normalize(0, 1)
])
data_module_myoarmband = MyoArmbandDataModule(
    batch_size=10000,
    num_workers=8,
    train_transforms=transform,
    val_transforms=transform,
    test_transforms=transform,
)
parameters = data_module_myoarmband.get_data_parameters()
model_myoarmband = OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height'])
classifier = Classifier(model_myoarmband, optim_kwargs={'lr': 0.001, 'weight_decay': 0.0001},
                        monitor='val_accuracy', sched_kwargs={'patience': 4, 'mode': 'max'})
logger_myoarmband = WandbLogger(project="EMG Armband", name="MyoArmband")
early_stop_callback = EarlyStopping(monitor='val_accuracy', patience=7, mode='max')
trainer_myoarmband = pl.Trainer(gpus=-1, max_epochs=100, logger=logger_myoarmband, accelerator="gpu",
                                callbacks=[early_stop_callback])

In [ ]:
trainer_myoarmband.fit(model=classifier, datamodule=data_module_myoarmband)

In [ ]:
trainer_myoarmband.test(model=classifier, datamodule=data_module_myoarmband)

In [ ]:
torch.save(model_capgmyo_chinese.state_dict(), os.path.join(MODELS_FOLDER, 'myoarmband.pt'))

## XGB Experiments

#### XGB CapgMyo

In [ ]:
logger_xgb_capgmyo = WandbLogger(project="EMG Armband", name="CapgMyoXGB")
adjusted_model = model_capgmyo
adjusted_model.model = adjusted_model.model[0:26]
data_module_xgb = CapgMyoDataModule(
    batch_size=1000,
    num_workers=3
)
xgb_classifier = LightningXGBClassifier(adjusted_model, data_module_xgb.num_classes, monitor='val_accuracy')
trainer_xgb = pl.Trainer(max_epochs=2, accelerator="cpu", logger=logger_xgb_capgmyo, num_sanity_val_steps=0)

In [ ]:
trainer_xgb.fit(model=xgb_classifier, datamodule=data_module_xgb)

In [ ]:
trainer_xgb.test(model=xgb_classifier, datamodule=data_module_xgb)

In [ ]:
xgb_classifier.xgbmodel.save_model(os.path.join(MODELS_FOLDER, "xgb_capgmyo.json"))

#### XGB Ninapro

In [ ]:
logger_xgb_ninapro = WandbLogger(project="EMG Armband", name="NinaproXGB")
adjusted_model = model_ninapro
adjusted_model.model = adjusted_model.model[0:26]
data_module_xgb = NinaProDataModule(
    batch_size=1000,
    num_workers=3
)
xgb_classifier = LightningXGBClassifier(adjusted_model, data_module_xgb.num_classes)
trainer_xgb = pl.Trainer(max_epochs=2, accelerator="cpu", logger=logger_xgb_ninapro, num_sanity_val_steps=0)

In [ ]:
trainer_xgb.fit(model=xgb_classifier, datamodule=data_module_xgb)

In [ ]:
trainer_xgb.test(model=xgb_classifier, datamodule=data_module_xgb)

In [ ]:
xgb_classifier.xgbmodel.save_model(os.path.join(MODELS_FOLDER, "xgb_ninapro.json"))

#### XGB MyoArmband

In [ ]:
logger_xgb_myoarmband = WandbLogger(project="EMG Armband", name="MyoArmbandXGB")
adjusted_model = model_myoarmband
adjusted_model.model = adjusted_model.model[0:26]
data_module_xgb = MyoArmbandDataModule(
    batch_size=1000,
    num_workers=3
)
xgb_classifier = LightningXGBClassifier(adjusted_model, data_module_xgb.num_classes)
trainer_xgb = pl.Trainer(max_epochs=2, accelerator="cpu", logger=logger_xgb_myoarmband, num_sanity_val_steps=0)

In [ ]:
trainer_xgb.fit(model=xgb_classifier, datamodule=data_module_xgb)

In [ ]:
trainer_xgb.test(model=xgb_classifier, datamodule=data_module_xgb)

In [ ]:
xgb_classifier.xgbmodel.save_model(os.path.join(MODELS_FOLDER, "xgb_myoarmband.json"))